# Pizza Huts Sales Forecasting Model

In [139]:
# Note from previous dev: when have chance use notebook markdown cells for better documentation and better labels, see cheat sheet https://www.markdownguide.org/cheat-sheet/

In [140]:
# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

In [141]:
# Previous Dev Notes: graphing functions from old project should suit the job, maybe need adjusting to your solution to the problem

# Plotting just a series with options to have next one after series and predicted value
def plot_series(series, nSteps, y=None, yPred=None, xLabel="$t$", yLabel="$x(t)$", legend=True):
    plt.plot(series, ".-")
    if y is not None:
        plt.plot(nSteps, y, "bo", label="Target")
    if yPred is not None:
        plt.plot(nSteps, yPred, "rx", markersize=10, label="Prediction")
    plt.grid(True)
    if xLabel:
        plt.xlabel(xLabel, fontsize=16)
    if yLabel:
        plt.ylabel(yLabel, fontsize=16, rotation=0)
    plt.hlines(0, 0, 100, linewidth=1)
    plt.axis([0, nSteps + 1, -1, 1])
    if legend and (y or yPred):
        plt.legend(fontsize=14, loc="upper left")

# Plotting ahead multiple times        
def plot_multiple_forecasts(X, Y, yPred, nSteps):
    n_steps = X.shape[1]
    ahead = Y.shape[1]
    plot_series(X[0, :, 0],nSteps)
    plt.plot(np.arange(n_steps, n_steps + ahead), Y[0, :, 0], "bo-", label="Actual")
    plt.plot(np.arange(n_steps, n_steps + ahead), yPred[0, :, 0], "rx-", label="Forecast", markersize=10)
    plt.axis([0, n_steps + ahead, -1, 1])
    plt.legend(fontsize=14)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

# Call when after creating a graph but before using plt.show() with the fig_id being name of file
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

This makes the code reproduce same results when using random determined code, alter the number to change the results. Also, check if a function has a random_state parameter, if it does add a number to make it reproduce the same results.
```py
np.random.seed(42)
tf.random.set_seed(42)
```

### Help reproduce same results, alter the number to change the results
np.random.seed(42)
tf.random.set_seed(42)

### Note from previous dev: place holder model change when start training
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

### Provide extra information during training
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

### Adam optimizers default learning rate = 0.001
model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=0.001),metrics=[last_time_step_mse])

### Note from previous dev: Change X_train, y_train & X_valid, y_vaild to name of the series or x y arrays you create with json file
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

## Task 1

#### Load JSON File

In [142]:
DATAPATH = "placement_interview_task_data.json"

In [143]:
# Read the JSON file
with open(DATAPATH) as json_file:
    data = json.load(json_file)

In [144]:
first_50_months = np.array([list(store.values())[:50] for store in data.values()])
first_50_months = np.expand_dims(first_50_months, axis=-1)
first_50_months.shape

(15000, 50, 1)

In [145]:
sale_51st_month = np.array([list(store.values())[50] for store in data.values()])
sale_51st_month = np.expand_dims(sale_51st_month, axis=-1)
sale_51st_month.shape

(15000, 1)

## Task 2